In [1]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
# from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_extra_data_embedding as training_testing_gru_extra_data_embedding

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")


os.environ["WANDB_NOTEBOOK_NAME"] = "GRU - v8.0.1 Topaz.ipynb"

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
torch.__version__

'2.1.1'

In [5]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
import rnn_tools.model_saver
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
# states = ["VIC"]
# states = ["NSW"]
states = ["NZ"]
# states = ['VIC', "SA"]
extra_tracks = ["Wentworth Park", "Wagga", "Temora"]
only_tracks = ["Bendigo", "Geelong", "Traralgon", "Ballarat", "Shepparton", "Warragul","Healesville"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
# data_file = './data/topaz_data_w_bsp_new4_small.fth'
data_file = './data/topaz_data_w_bsp_new4.fth'
# data_file = './data/topaz_data_w_bsp_new2_nz.fth'
# data_file = './data/topaz_data_w_bsp_new2.fth'
# data_file = './data/topaz_data_w_bsp_new_w_price.fth'
# data_file = './data/topaz_data_w_bsp.fth'
# data_file = './data/gru_inputs_kitchen_sink_topaz.fth'
raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,only_tracks=only_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='boosted_sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python


c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\raceDB.py:421: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dog_stats_df['stats'] = dog_stats_df['stats_topaz']


(2520524, 146)
54
Latest date = 2024-05-19 00:00:00
size after state filter (2520524, 147)
(390887, 148)
Index(['index', 'meetingDate', 'state', 'track_name', 'dist', 'raceId',
       'race_grade', 'race_num', 'box', 'rugNumber',
       ...
       'next_race', 'prev_start_price', 'stats_topaz', 'dogid', 'raceid',
       'stats_cols', 'track_hash', 'stats', 'race_time', 'stats_cuda'],
      dtype='object', length=149)
state  track_name  
NZ     Addington       84612
       Ascot Park      14493
       Ashburton          94
       Cambridge       21519
       Forbury Park     6187
       Hatrick         30300
       Manawatu        28593
       Manukau         23981
       Taranaki          235
       Tokoroa           179
Name: count, dtype: int64
Latest date = 2024-05-16 00:00:00
stats = [ 0.00000000e+00  1.00000000e+00  1.35416672e-01  0.00000000e+00
  0.00000000e+00  1.48286432e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00

100%|██████████| 25/25 [00:00<?, ?it/s]


race_ids=['397536987', '397536988', '397536989', '397536993', '397536992', '397536990', '397536991', '397536994', '397536998', '397536995', '397536996', '397536997', '397537055', '397537056', '397537057', '397537060', '397537058', '397537059', '397537065', '397537061', '397537062', '397537064', '397537063', '397624250', '397537066', '397624270', '397624269', '397624268', '397624265', '397624266', '397624267', '397624256', '397624253', '397624263', '397624259', '397624246', '397624212', '397624211', '397624339', '397624214', '397624213', '397624217', '397624215', '397624216', '397624218', '397624220', '397624221', '397624329', '397624219', '397624222', '397624340', '397624334', '397624330', '397624331', '397624326', '397624328', '397624327', '397624338', '397624337', '397624332', '397624333', '397624335', '397624336', '397624319', '397624325', '397624324', '397624323', '397624322', '397624321', '397624320', '397624316', '397624318', '397624317', '397624315', '397624314', '399450852', '3

c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:419: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.prob = torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)


number of races = 27811, number of unique dogs = 4978
0          (Hatrick, 305)
1          (Hatrick, 305)
2          (Hatrick, 305)
3          (Hatrick, 305)
4          (Hatrick, 520)
               ...       
27806    (Addington, 295)
27807    (Addington, 295)
27808    (Addington, 645)
27809    (Addington, 520)
27810    (Addington, 295)
Length: 27811, dtype: object


In [6]:
feature_cols = ['dam_finishingPlaceMovement_max_28D',
 'dam_finishingPlaceMovement_max_365D',
 'dam_finishingPlaceMovement_max_91D',
 'dam_finishingPlaceMovement_mean_28D',
 'dam_finishingPlaceMovement_mean_365D',
 'dam_finishingPlaceMovement_mean_91D',
 'dam_finishingPlaceMovement_median_28D',
 'dam_finishingPlaceMovement_median_365D',
 'dam_finishingPlaceMovement_median_91D',
 'dam_finishingPlaceMovement_min_28D',
 'dam_finishingPlaceMovement_min_365D',
 'dam_finishingPlaceMovement_min_91D',
 'dam_finishingPlaceMovement_std_28D',
 'dam_finishingPlaceMovement_std_365D',
 'dam_finishingPlaceMovement_std_91D',
 'dam_marginLog_max_28D',
 'dam_marginLog_max_365D',
 'dam_marginLog_max_91D',
 'dam_marginLog_mean_28D',
 'dam_marginLog_mean_365D',
 'dam_marginLog_mean_91D',
 'dam_marginLog_median_28D',
 'dam_marginLog_median_365D',
 'dam_marginLog_median_91D',
 'dam_marginLog_min_28D',
 'dam_marginLog_min_365D',
 'dam_marginLog_min_91D',
 'dam_marginLog_std_28D',
 'dam_marginLog_std_365D',
 'dam_marginLog_std_91D',
 'dam_placeLog_max_28D',
 'dam_placeLog_max_365D',
 'dam_placeLog_max_91D',
 'dam_placeLog_mean_28D',
 'dam_placeLog_mean_365D',
 'dam_placeLog_mean_91D',
 'dam_placeLog_median_28D',
 'dam_placeLog_median_365D',
 'dam_placeLog_median_91D',
 'dam_placeLog_min_28D',
 'dam_placeLog_min_365D',
 'dam_placeLog_min_91D',
 'dam_placeLog_std_28D',
 'dam_placeLog_std_365D',
 'dam_placeLog_std_91D',
 'dam_prizemoneyLog_max_28D',
 'dam_prizemoneyLog_max_365D',
 'dam_prizemoneyLog_max_91D',
 'dam_prizemoneyLog_mean_28D',
 'dam_prizemoneyLog_mean_365D',
 'dam_prizemoneyLog_mean_91D',
 'dam_prizemoneyLog_median_28D',
 'dam_prizemoneyLog_median_365D',
 'dam_prizemoneyLog_median_91D',
 'dam_prizemoneyLog_min_28D',
 'dam_prizemoneyLog_min_365D',
 'dam_prizemoneyLog_min_91D',
 'dam_prizemoneyLog_std_28D',
 'dam_prizemoneyLog_std_365D',
 'dam_prizemoneyLog_std_91D',
 'dam_runTimeNorm_max_28D',
 'dam_runTimeNorm_max_365D',
 'dam_runTimeNorm_max_91D',
 'dam_runTimeNorm_mean_28D',
 'dam_runTimeNorm_mean_365D',
 'dam_runTimeNorm_mean_91D',
 'dam_runTimeNorm_median_28D',
 'dam_runTimeNorm_median_365D',
 'dam_runTimeNorm_median_91D',
 'dam_runTimeNorm_min_28D',
 'dam_runTimeNorm_min_365D',
 'dam_runTimeNorm_min_91D',
 'dam_runTimeNorm_std_28D',
 'dam_runTimeNorm_std_365D',
 'dam_runTimeNorm_std_91D',
 'dog_finishingPlaceMovement_max_28D',
 'dog_finishingPlaceMovement_max_365D',
 'dog_finishingPlaceMovement_max_91D',
 'dog_finishingPlaceMovement_mean_28D',
 'dog_finishingPlaceMovement_mean_365D',
 'dog_finishingPlaceMovement_mean_91D',
 'dog_finishingPlaceMovement_median_28D',
 'dog_finishingPlaceMovement_median_365D',
 'dog_finishingPlaceMovement_median_91D',
 'dog_finishingPlaceMovement_min_28D',
 'dog_finishingPlaceMovement_min_365D',
 'dog_finishingPlaceMovement_min_91D',
 'dog_finishingPlaceMovement_std_28D',
 'dog_finishingPlaceMovement_std_365D',
 'dog_finishingPlaceMovement_std_91D',
 'dog_marginLog_max_28D',
 'dog_marginLog_max_365D',
 'dog_marginLog_max_91D',
 'dog_marginLog_mean_28D',
 'dog_marginLog_mean_365D',
 'dog_marginLog_mean_91D',
 'dog_marginLog_median_28D',
 'dog_marginLog_median_365D',
 'dog_marginLog_median_91D',
 'dog_marginLog_min_28D',
 'dog_marginLog_min_365D',
 'dog_marginLog_min_91D',
 'dog_marginLog_std_28D',
 'dog_marginLog_std_365D',
 'dog_marginLog_std_91D',
 'dog_placeLog_max_28D',
 'dog_placeLog_max_365D',
 'dog_placeLog_max_91D',
 'dog_placeLog_mean_28D',
 'dog_placeLog_mean_365D',
 'dog_placeLog_mean_91D',
 'dog_placeLog_median_28D',
 'dog_placeLog_median_365D',
 'dog_placeLog_median_91D',
 'dog_placeLog_min_28D',
 'dog_placeLog_min_365D',
 'dog_placeLog_min_91D',
 'dog_placeLog_std_28D',
 'dog_placeLog_std_365D',
 'dog_placeLog_std_91D',
 'dog_prizemoneyLog_max_28D',
 'dog_prizemoneyLog_max_365D',
 'dog_prizemoneyLog_max_91D',
 'dog_prizemoneyLog_mean_28D',
 'dog_prizemoneyLog_mean_365D',
 'dog_prizemoneyLog_mean_91D',
 'dog_prizemoneyLog_median_28D',
 'dog_prizemoneyLog_median_365D',
 'dog_prizemoneyLog_median_91D',
 'dog_prizemoneyLog_min_28D',
 'dog_prizemoneyLog_min_365D',
 'dog_prizemoneyLog_min_91D',
 'dog_prizemoneyLog_std_28D',
 'dog_prizemoneyLog_std_365D',
 'dog_prizemoneyLog_std_91D',
 'dog_runTimeNorm_max_28D',
 'dog_runTimeNorm_max_365D',
 'dog_runTimeNorm_max_91D',
 'dog_runTimeNorm_mean_28D',
 'dog_runTimeNorm_mean_365D',
 'dog_runTimeNorm_mean_91D',
 'dog_runTimeNorm_median_28D',
 'dog_runTimeNorm_median_365D',
 'dog_runTimeNorm_median_91D',
 'dog_runTimeNorm_min_28D',
 'dog_runTimeNorm_min_365D',
 'dog_runTimeNorm_min_91D',
 'dog_runTimeNorm_std_28D',
 'dog_runTimeNorm_std_365D',
 'dog_runTimeNorm_std_91D',
 'sire_finishingPlaceMovement_max_28D',
 'sire_finishingPlaceMovement_max_365D',
 'sire_finishingPlaceMovement_max_91D',
 'sire_finishingPlaceMovement_mean_28D',
 'sire_finishingPlaceMovement_mean_365D',
 'sire_finishingPlaceMovement_mean_91D',
 'sire_finishingPlaceMovement_median_28D',
 'sire_finishingPlaceMovement_median_365D',
 'sire_finishingPlaceMovement_median_91D',
 'sire_finishingPlaceMovement_min_28D',
 'sire_finishingPlaceMovement_min_365D',
 'sire_finishingPlaceMovement_min_91D',
 'sire_finishingPlaceMovement_std_28D',
 'sire_finishingPlaceMovement_std_365D',
 'sire_finishingPlaceMovement_std_91D',
 'sire_marginLog_max_28D',
 'sire_marginLog_max_365D',
 'sire_marginLog_max_91D',
 'sire_marginLog_mean_28D',
 'sire_marginLog_mean_365D',
 'sire_marginLog_mean_91D',
 'sire_marginLog_median_28D',
 'sire_marginLog_median_365D',
 'sire_marginLog_median_91D',
 'sire_marginLog_min_28D',
 'sire_marginLog_min_365D',
 'sire_marginLog_min_91D',
 'sire_marginLog_std_28D',
 'sire_marginLog_std_365D',
 'sire_marginLog_std_91D',
 'sire_placeLog_max_28D',
 'sire_placeLog_max_365D',
 'sire_placeLog_max_91D',
 'sire_placeLog_mean_28D',
 'sire_placeLog_mean_365D',
 'sire_placeLog_mean_91D',
 'sire_placeLog_median_28D',
 'sire_placeLog_median_365D',
 'sire_placeLog_median_91D',
 'sire_placeLog_min_28D',
 'sire_placeLog_min_365D',
 'sire_placeLog_min_91D',
 'sire_placeLog_std_28D',
 'sire_placeLog_std_365D',
 'sire_placeLog_std_91D',
 'sire_prizemoneyLog_max_28D',
 'sire_prizemoneyLog_max_365D',
 'sire_prizemoneyLog_max_91D',
 'sire_prizemoneyLog_mean_28D',
 'sire_prizemoneyLog_mean_365D',
 'sire_prizemoneyLog_mean_91D',
 'sire_prizemoneyLog_median_28D',
 'sire_prizemoneyLog_median_365D',
 'sire_prizemoneyLog_median_91D',
 'sire_prizemoneyLog_min_28D',
 'sire_prizemoneyLog_min_365D',
 'sire_prizemoneyLog_min_91D',
 'sire_prizemoneyLog_std_28D',
 'sire_prizemoneyLog_std_365D',
 'sire_prizemoneyLog_std_91D',
 'sire_runTimeNorm_max_28D',
 'sire_runTimeNorm_max_365D',
 'sire_runTimeNorm_max_91D',
 'sire_runTimeNorm_mean_28D',
 'sire_runTimeNorm_mean_365D',
 'sire_runTimeNorm_mean_91D',
 'sire_runTimeNorm_median_28D',
 'sire_runTimeNorm_median_365D',
 'sire_runTimeNorm_median_91D',
 'sire_runTimeNorm_min_28D',
 'sire_runTimeNorm_min_365D',
 'sire_runTimeNorm_min_91D',
 'sire_runTimeNorm_std_28D',
 'sire_runTimeNorm_std_365D',
 'sire_runTimeNorm_std_91D',
 'trainer_finishingPlaceMovement_max_28D',
 'trainer_finishingPlaceMovement_max_365D',
 'trainer_finishingPlaceMovement_max_91D',
 'trainer_finishingPlaceMovement_mean_28D',
 'trainer_finishingPlaceMovement_mean_365D',
 'trainer_finishingPlaceMovement_mean_91D',
 'trainer_finishingPlaceMovement_median_28D',
 'trainer_finishingPlaceMovement_median_365D',
 'trainer_finishingPlaceMovement_median_91D',
 'trainer_finishingPlaceMovement_min_28D',
 'trainer_finishingPlaceMovement_min_365D',
 'trainer_finishingPlaceMovement_min_91D',
 'trainer_finishingPlaceMovement_std_28D',
 'trainer_finishingPlaceMovement_std_365D',
 'trainer_finishingPlaceMovement_std_91D',
 'trainer_marginLog_max_28D',
 'trainer_marginLog_max_365D',
 'trainer_marginLog_max_91D',
 'trainer_marginLog_mean_28D',
 'trainer_marginLog_mean_365D',
 'trainer_marginLog_mean_91D',
 'trainer_marginLog_median_28D',
 'trainer_marginLog_median_365D',
 'trainer_marginLog_median_91D',
 'trainer_marginLog_min_28D',
 'trainer_marginLog_min_365D',
 'trainer_marginLog_min_91D',
 'trainer_marginLog_std_28D',
 'trainer_marginLog_std_365D',
 'trainer_marginLog_std_91D',
 'trainer_placeLog_max_28D',
 'trainer_placeLog_max_365D',
 'trainer_placeLog_max_91D',
 'trainer_placeLog_mean_28D',
 'trainer_placeLog_mean_365D',
 'trainer_placeLog_mean_91D',
 'trainer_placeLog_median_28D',
 'trainer_placeLog_median_365D',
 'trainer_placeLog_median_91D',
 'trainer_placeLog_min_28D',
 'trainer_placeLog_min_365D',
 'trainer_placeLog_min_91D',
 'trainer_placeLog_std_28D',
 'trainer_placeLog_std_365D',
 'trainer_placeLog_std_91D',
 'trainer_prizemoneyLog_max_28D',
 'trainer_prizemoneyLog_max_365D',
 'trainer_prizemoneyLog_max_91D',
 'trainer_prizemoneyLog_mean_28D',
 'trainer_prizemoneyLog_mean_365D',
 'trainer_prizemoneyLog_mean_91D',
 'trainer_prizemoneyLog_median_28D',
 'trainer_prizemoneyLog_median_365D',
 'trainer_prizemoneyLog_median_91D',
 'trainer_prizemoneyLog_min_28D',
 'trainer_prizemoneyLog_min_365D',
 'trainer_prizemoneyLog_min_91D',
 'trainer_prizemoneyLog_std_28D',
 'trainer_prizemoneyLog_std_365D',
 'trainer_prizemoneyLog_std_91D',
 'trainer_runTimeNorm_max_28D',
 'trainer_runTimeNorm_max_365D',
 'trainer_runTimeNorm_max_91D',
 'trainer_runTimeNorm_mean_28D',
 'trainer_runTimeNorm_mean_365D',
 'trainer_runTimeNorm_mean_91D',
 'trainer_runTimeNorm_median_28D',
 'trainer_runTimeNorm_median_365D',
 'trainer_runTimeNorm_median_91D',
 'trainer_runTimeNorm_min_28D',
 'trainer_runTimeNorm_min_365D',
 'trainer_runTimeNorm_min_91D',
 'trainer_runTimeNorm_std_28D',
 'trainer_runTimeNorm_std_365D',
 'trainer_runTimeNorm_std_91D',
 'dogAgeScaled',
 'lastFiveWinPercentage',
 'lastFivePlacePercentage',
 'weightInKgScaled',
 'rolling_box_win_percentage']

In [7]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def load_matching_state_dict(model, state_dict):
    model_dict = model.state_dict()

    # Filter out unnecessary keys
    matching_keys = {k: v for k, v in state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    excluded_keys = set(state_dict.keys()) - set(matching_keys.keys())

    # Overwrite entries in the existing state dict
    model_dict.update(matching_keys)

    # Load the new state dict
    model.load_state_dict(model_dict)

    print("Loaded keys:")
    for key in matching_keys:
        print(key)

    print("\nExcluded keys:")
    for key in excluded_keys:
        print(key)

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Reporting", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*6+reg_stat_mask+[0]*80
      # stat_mask = [1]*30
      stat_mask = [1]*18+36*[0]
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box,T_box, T_dist
      data_mask =  [1]*18+36*[1]
      # data_mask =  [1]*30
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      print(f"{data_mask=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-06-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra_embedding(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        load_matching_state_dict(model, model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        model = model.to(device)        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      try:
        return training_testing_gru_extra_data_embedding.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      raceDB.create_hidden_states_dict_v2()
      rnn_tools.model_saver.model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")


    return (model,dataset, optimizer)

In [8]:
raceDB.batches_setup = False

In [9]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':0,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':0,
    '_pos_out_avg_1':0,
    '_post_change_avg_1':0,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':0,
    '_last_start_prob':0,
}

wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.0001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2022-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 450,
    'stat_list_dict': stat_list_dict,
}

In [10]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])
torch.cuda.empty_cache()
raceDB.race_prices_to_prob()
print(raceDB.latest_date)
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  
    race.loss_log = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

for race in raceDB.racesDict.values():
    if race.classes.isnan().sum():
        print(race.classes.sum())
        print(race.raceid)
        race.classes = race.classes.nan_to_num(0,0,0)

2024-05-16 00:00:00
tensor(nan, device='cuda:0')
402888097
tensor(nan, device='cuda:0')
458549780
tensor(nan, device='cuda:0')
618363326


In [11]:
torch.cuda.empty_cache()   

In [12]:
x = torch.rand(10,8)
x.argmax(dim=-1)

tensor([4, 0, 1, 3, 1, 7, 3, 7, 0, 3])

In [14]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
importlib.reload(training_testing_gru_extra_data_embedding)
importlib.reload(rnn_tools.model_saver)
# stop
# importlib.reload(training_testing_gru_extra_data_state_batch)
# importlib.reload(training_testing_gru_extra_data_toasty)
# importlib.reload(training_testing_gru_extra_data_toasty_og)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC SA NSW-ancient-valley-726', prev_model_version = 280) 
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC-daily-sunset-633', prev_model_version = 700)
model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NZ-glad-river-548', prev_model_version = 165)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'hidden_size': 256, 'stats': "['dogAgeScaled', 'boxNumber', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_distance_mean_1', 'dog_boxNumber_mean_1', 'dog_runTimeNorm_mean_1', 'dog_place_mean_1', 'dog_resultMargin_mean_1', 'dog_split_time_margin_mean_1', 'dog_split_runTimeNorm_mean_1', 'dog_time_1_mean_1', 'dog_run_home_TimeNorm_mean_1', 'dog_finishingPlaceMovement_mean_1', 'dog_averageSpeed_mean_1', 'dog_win_mean_1', 'trainer_distance_mean_365D', 'trainer_boxNumber_mean_365D', 'trainer_runTimeNorm_mean_365D', 'trainer_place_mean_365D', 'trainer_resultMargin_mean_365D', 'trainer_split_time_margin_mean_365D', 'trainer_split_runTimeNorm_mean_365D', 'trainer_time_1_mean_365D', 'trainer_run_home_TimeNorm_mean_365D', 'trainer_finishingPlaceMovement_mean_365D', 'trainer_averageSpeed_mean_365D', 'trainer_win_mean_365D', 'dam_distance_mean_365D', 'dam_boxNumber_mean_365D', 'dam_runTimeNorm_mean_365D', 'dam_place_mean_365D', 'dam_resu

C:\Users\Nick\AppData\Local\Temp\ipykernel_32828\1132403399.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)
c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loading model NZ-glad-river-548, version 165
odict_keys(['h0', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'gru.weight_ih_l1', 'gru.weight_hh_l1', 'gru.bias_ih_l1', 'gru.bias_hh_l1', 'fc0.weight', 'fc0.bias', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'batch_norm_data.weight', 'batch_norm_data.bias', 'batch_norm_data.running_mean', 'batch_norm_data.running_var', 'batch_norm_data.num_batches_tracked', 'track_embedding.weight', 'extra_1.batch_norm.weight', 'extra_1.batch_norm.bias', 'extra_1.batch_norm.running_mean', 'extra_1.batch_norm.running_var', 'extra_1.batch_norm.num_batches_tracked', 'extra_1.fc0_p1.weight', 'extra_1.fc0_p1.bias', 'extra_1.fc0_p2.weight', 'extra_1.fc0_p2.bias', 'extra_1.fc0_p3.weight', 'extra_1.fc0_p3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
Loaded keys:
h0
gru.weight_ih_l0
gru.weight_hh_l0
gru.bias_ih_l0

  0%|          | 0/2500 [00:00<?, ?it/s]

Quickpass time 3.041828399989754
Test val pass Time: 3.894775799999479
val val pass Time: 2.596637799986638
Test time : 18.630354799999623, Val time : 7.543352300010156, Total time : 26.17370710000978
New Max ROI: -0.0424, 0.0011, 1.8063
created path


...

 21%|██        | 520/2500 [2:20:17<8:05:51, 14.72s/it]

Quickpass time 3.1771814999810886
Test val pass Time: 5.617938899988076
val val pass Time: 3.036266799987061
Test time : 19.711657400010154, Val time : 8.099848999991082, Total time : 27.811506400001235
New Max ROI: 0.0239, 0.0168, 1.74368


 21%|██        | 526/2500 [2:22:22<9:27:01, 17.23s/it] 

Quickpass time 4.477521799999522


 21%|██        | 530/2500 [2:23:20<8:25:00, 15.38s/it]